# Sums of three cubes: finding the first five examples

**Problem.** Print the first five cubes $n^3$ that can be written as a sum of three positive cubes:

$$n^3 = a^3 + b^3 + c^3$$

with integers $1 \le a \le b \le c < n$.

This notebook demonstrates an efficient search strategy using a dictionary (hash table).

## Key idea: reduce a 3-variable search to 2 variables

Rewrite:

$$c^3 = n^3 - a^3 - b^3$$

For each $n$:
1. Precompute cubes $k^3$ for $k=1..n-1$.
2. Store a dictionary mapping **cube value → base**.
3. Loop over pairs $(a,b)$, compute the needed value, and look it up.
4. Enforce ordering ($a\le b\le c$) and $c<n$.

This gives an $O(n^2)$ search per $n$ (average-case $O(1)$ lookups).

In [6]:
def first_five_sum_of_three_cubes(limit_n: int = 1000) -> None:
    """
    Print the first five cubes n^3 that can be written as a^3 + b^3 + c^3
    with positive integers 1 <= a <= b <= c < n.

    For each n:
      - Precompute cubes up to n-1
      - Build a dict cube_value -> base for fast lookup
      - Loop over (a,b) and solve for c via hash lookup
    """
    
    found = 0
    # Counter for how many valid representations we have found so far

    for n in range(2, limit_n + 1):
        # Try values of n starting from 2 up to limit_n

        target = n * n * n
        # Compute n^3 (the value we want to write as a sum of three cubes)

        cubes = [k * k * k for k in range(n)]
        # Precompute all cubes k^3 for k = 0,1,...,n-1
        # This avoids recomputing k**3 repeatedly later

        cube_to_base = {cubes[k]: k for k in range(1, n)}
        # Build a mapping cube value -> base
        # Example: 125 -> 5
        # We start from k=1 because a,b,c must be positive

        rep = None
        # Starts as None (meaning "not found yet")

        for a in range(1, n):
        # Loop over possible values of a (1 <= a < n)

            a3 = cubes[a]
            # Store a^3 once to avoid recomputing it

            for b in range(a, n):
            # Loop over b starting from a to enforce a <= b

                needed = target - (a3 + cubes[b])
                # Rearranged equation:
                # c^3 = n^3 - a^3 - b^3
                # So "needed" is the value c^3 would have to equal

                c = cube_to_base.get(needed)
                # Look up whether "needed" is a perfect cube < n
                # Returns the corresponding base c if it exists,
                # otherwise returns None

                if c is None:
                    # If needed is not a cube, skip to next b
                    continue

                if c < b:
                    # Enforce ordering b <= c
                    # If violated, skip this case
                    continue

                rep = (a, b, c)
                # Store the valid triple

                break
                # Stop searching over b once one valid triple is found

            if rep:
                # If we already found a triple for this n,
                # stop searching over a as well
                break

        if rep:
        # If a representation was found for this n:

            a, b, c = rep
            # Unpack the triple

            found += 1
            # Increment the count of successful n values

            print(f"{found}. {n}^3 = {target} = {a}^3 + {b}^3 + {c}^3")
            # Print the result in formatted form

            if found == 5:
                # Stop completely once we have found five examples
                return

    raise RuntimeError(f"Only found {found} up to n={limit_n}; increase limit_n.")
    # If we exit the loop without finding five examples,
    # raise an error suggesting a larger search range


## Run it

In [ ]:
first_five_sum_of_three_cubes(limit_n=1000)

## Notes

- The dictionary works because $k^3$ is strictly increasing for positive integers, so each cube value maps to a unique base.
- The check `c < b` enforces $b \le c$.

### Complexity (searching up to N)
Worst-case work is roughly $\sum_{n=1}^{N} O(n^2) = O(N^3)$, but the program stops early once it finds 5 solutions.

In [ ]:
import time
t0 = time.time()
first_five_sum_of_three_cubes(limit_n=1000)
print(f"Elapsed: {time.time() - t0:.3f} s")